In [10]:
import pandas as pd
import numpy as np

In [126]:
query = pd.read_pickle('player_data.pkl')
prepared_query = query.format(season="2018", partition_date="20181025")
df = pd.read_gbq(prepared_query, project_id='scarlet-labs', dialect='standard')

Requesting query... ok.
Query running...
Query done.
Cache hit.

Retrieving results...
Got 24629 rows.

Total time taken 5.93 s.
Finished at 2018-10-31 18:56:20.


In [150]:
most_recent_df = df.groupby('bbrefID', as_index=False, group_keys=False).max()[['bbrefID', 'date']].reset_index().drop(['index'], axis=1)

In [117]:
def get_rolling_game_avgs(df, index_on, games=20):
    _df = df.groupby(index_on, as_index=False, group_keys=False).rolling(games).mean().reset_index().drop(["index"], axis=1).fillna(0)
    df_transformed = _df.set_index(["date"] + index_on).select_dtypes([np.number])
    new_col_names = ["{col}_{index_type}_{rolling}g".format(col=c,
                                                            index_type = '_'.join(index_on),
                                                            rolling=games) 
                     for c in df_transformed.columns]
    df_transformed.columns = new_col_names
    return df_transformed

In [118]:
player_feature_list = [get_rolling_game_avgs(df, index_on=['bbrefID'], games=g) for g in [5,10,20]]
opp_feature_list = [get_rolling_game_avgs(df, index_on=['opp', 'pos'], games=g) for g in [5,10,20]]

In [119]:
player_feature_df = pd.concat(player_feature_list, axis=1)
opp_feature_df = pd.concat(opp_feature_list, axis=1).reset_index().groupby(['date', 'opp', 'pos']).median()

In [139]:
tmp = df.set_index(['date', 'bbrefID']).join(player_feature_df).reset_index().set_index(['date', 'opp', 'pos']).join(opp_feature_df).reset_index()

In [152]:
tmp.join(most_recent_df)

ValueError: columns overlap but no suffix specified: Index(['bbrefID', 'date'], dtype='object')

In [156]:
most_recent_df.set_index(['bbrefID', 'date']).join(tmp.set_index(['bbrefID', 'date']))

,,opp,pos,player,tm,G,GS,venue,secs_played,plus_minus,dk,...,DRB_pct_opp_pos_20g,TRB_pct_opp_pos_20g,AST_pct_opp_pos_20g,STL_pct_opp_pos_20g,BLK_pct_opp_pos_20g,TOV_pct_opp_pos_20g,USG_pct_opp_pos_20g,ORtg_opp_pos_20g,DRtg_opp_pos_20g,GmSc_opp_pos_20g
bbrefID,date,,,,,,,,,,,,,,,,,,,,,
abrinal01,2018-04-09,MIA,SG,Alex Abrines,OKC,75,0,away,706,+10,2.75,...,11.0475,6.4700,10.0025,2.1825,0.8200,8.6725,20.3350,108.300,104.400,8.2575
acyqu01,2018-04-11,BOS,PF,Quincy Acy,BRK,70,0,away,1414,-6,16.25,...,21.5150,13.3650,9.9150,1.7050,2.3950,11.6500,17.9150,103.250,104.800,8.3000
adamsst01,2018-04-11,MEM,C,Steven Adams,OKC,76,1,home,2189,+17,20.75,...,30.4900,20.0125,11.9350,1.5275,2.5275,0.0000,17.4225,110.050,101.675,9.1600
adebaba01,2018-04-11,TOR,C,Bam Adebayo,MIA,69,0,home,1097,+16,12.50,...,23.2100,15.2050,10.6475,1.5100,3.0700,0.0000,17.8950,117.175,105.950,9.3350
afflaar01,2018-04-11,WAS,SG,Arron Afflalo,ORL,53,0,home,1027,+4,7.50,...,11.9750,6.7250,18.0925,1.2950,0.4150,6.3725,21.5300,117.000,108.525,8.5900
aldrico01,2018-04-09,MEM,C,Cole Aldrich,MIN,21,0,home,63,0,2.75,...,29.0350,19.4400,12.5750,2.0200,2.6300,0.0000,17.9000,111.750,98.800,7.6200
aldrila01,2018-04-11,NOP,C,LaMarcus Aldridge,SAS,75,1,away,1352,-16,13.75,...,19.2550,13.3500,10.3150,1.5250,1.9550,0.0000,16.6650,90.500,109.500,6.2900
allenja01,2018-04-11,BOS,C,Jarrett Allen,BRK,72,1,away,1160,-9,14.75,...,17.1975,12.8225,4.5200,1.3300,1.4300,0.0000,14.4575,106.100,106.100,5.8750
allenka01,2018-04-11,BRK,PG,Kadeem Allen,BOS,18,0,home,1468,-12,13.25,...,11.4500,6.5150,20.4850,1.3600,0.4400,0.0000,19.1550,111.400,111.250,7.9850
